# English to French prediction using encoder-decoder model.

In [ ]:
## import the needed libraries

In [2]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [3]:
## initialize the variables to use in our model
batch_size = 64  
epochs = 100  
latent_dim = 256  
num_samples = 10000

In [4]:
## mount the drive to google colab
from google.colab import drive

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
## extracted the zip file from colab
from zipfile import ZipFile

In [7]:
with ZipFile('/content/drive/MyDrive/fra-eng.zip','r') as zipobj:
   zipobj.extractall('/content/drive/MyDrive/frenchtoenglish')

In [8]:
data_path = '/content/drive/MyDrive/frenchtoenglish/fra.txt'

In [9]:
## open the text file
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [10]:
len(lines)

194514

In [11]:
## put each english word in a line to input_texts
## put each french word in a line to target_texts
## put each characters in english to input_characters
## put each characters in french to target_characters
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [12]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [13]:
## create a dictionary out of input_characters and target_characters
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [14]:
## one hot encode your all elements of input_texts and target_texts
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

In [15]:
## now use the above created np arrays and represesnt all ur datas in a one hot encoded form.
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [16]:
## create the encoder model
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [17]:
# Set up the decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,   initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [18]:
## define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [19]:
## train the created model with the encoded datas
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 58s 415ms/step - loss: 1.1291 - accuracy: 0.7372 - val_loss: 1.0235 - val_accuracy: 0.7264
Epoch 2/100
125/125 [==============================] - 52s 412ms/step - loss: 0.8075 - accuracy: 0.7821 - val_loss: 0.8098 - val_accuracy: 0.7776
Epoch 3/100
125/125 [==============================] - 52s 414ms/step - loss: 0.6545 - accuracy: 0.8162 - val_loss: 0.6991 - val_accuracy: 0.7960
Epoch 4/100
125/125 [==============================] - 50s 401ms/step - loss: 0.5733 - accuracy: 0.8331 - val_loss: 0.6311 - val_accuracy: 0.8159
Epoch 5/100
125/125 [==============================] - 52s 414ms/step - loss: 0.5248 - accuracy: 0.8467 - val_loss: 0.5816 - val_accuracy: 0.8334
Epoch 6/100
125/125 [==============================] - 50s 401ms/step - loss: 0.4892 - accuracy: 0.8563 - val_loss: 0.5709 - val_accuracy: 0.8329
Epoch 7/100
125/125 [==============================] - 50s 402ms/step - loss: 0.4619 - accuracy: 0.8633 - val_loss: 0.5346 -

In [20]:
## take encoder
encoder_model = Model(encoder_inputs, encoder_states)


In [21]:
## take decoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [22]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [23]:
## function used to predict the french equivalent of english sentence passed in one hot encoded form.
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

In [25]:
## make predictions on first 100 english words and print
## its predicted french equivalent.
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Bouge !

-
Input sentence: Go.
Decoded sentence: Bouge !

-
Input sentence: Go.
Decoded sentence: Bouge !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run.
Decoded sentence: Fuyons !

-
Input sentence: Run.
Decoded sentence: Fuyons !

-
Input sentence: Run.
Decoded sentence: Fuyons !

-
Input sentence: Run.
Decoded sentence: Fuyons !

-
Input sentence: Run.
Decoded sentence: Fuyons !

-
Input sentence: Run.
Decoded sentence: Fuyons !

-
Input sentence: Run.
Decoded sentence: 